In [46]:
#import libraries
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
import tensorflow as tf
from multiprocessing import Queue

import matplotlib.pyplot as plt

In [47]:
train_path = r"C:\Users\Goba-PC\Desktop\python\obrabotka\fishes\train"
valid_path = r"C:\Users\Goba-PC\Desktop\python\obrabotka\fishes\valid"
test_path = r"C:\Users\Goba-PC\Desktop\python\obrabotka\fishes\test"

In [48]:
img_size = (224,224)
batch_size = 64

In [49]:
#Creating generator for Training DataSet
train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical')

#Creating generator for Validation DataSet
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_datagen.flow_from_directory(
        valid_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical')

#Creating generator for Test DataSet
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 58388 images belonging to 400 classes.
Found 2000 images belonging to 400 classes.
Found 2000 images belonging to 400 classes.


In [50]:
base_model=InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3))

In [51]:
#freeze the base model
base_model.trainable = False

In [52]:
#Create new model on top
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(400,activation='softmax',kernel_initializer='glorot_normal'))

In [53]:
#Train the model on new data.
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_generator,epochs=40,validation_data=val_generator,workers=10)

Epoch 1/40
 81/913 [=>............................] - ETA: 30:05 - loss: 102.0558 - accuracy: 0.0035

KeyboardInterrupt: 

In [ ]:
#Some visualizations
#Loss
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.legend()
plt.show()
#Accuracy
plt.plot(history.history['accuracy'],label='acc')
plt.plot(history.history['val_accuracy'],label='val_acc')
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_generator,use_multiprocessing=True,workers=10)